<a href="https://colab.research.google.com/github/Blickmeister/Blickmeister-Corona_app_neural_network_model/blob/main/SMAP_projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importy
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from keras.utils import plot_model
from tensorflow.keras.models import load_model

Předzpracování a analýza dat:
Tato fáze spočívá v získání dat, očistění od nepotřebných údajů, vymezení období sledování, seskupení dat dle krajů a v následné analýze pro podporu dalších kroků.

In [ ]:
# získání dat ze serveru
data = pd.read_csv("https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/kraj-okres-nakazeni-vyleceni-umrti.csv")
data.tail()

In [ ]:
# sledujeme data od 1.8.2020 do 20.11.2020
data_in_period = data.loc[(data['datum'] >= '2020-08-01') & (data['datum'] <= '2020-11-20')]

# počet vyléčených a mrtvých nás nezajímají
number_of_cases_all = data_in_period.drop(columns=['kumulativni_pocet_vylecenych','kumulativni_pocet_umrti'])

# sjednocení dle krajů a dnů
number_of_cases_by_regions_and_date = number_of_cases_all.groupby(['kraj_nuts_kod','datum'], as_index=False)[['kumulativni_pocet_nakazenych']].sum()

# rozdělení do krajů
region_codes = ["CZ010", "CZ031", "CZ064", "CZ041", "CZ052", "CZ051", "CZ080",
            "CZ071", "CZ053", "CZ032", "CZ020", "CZ042", "CZ063", "CZ072"]

# funkce pro získání jména kraje na základě jeho kódu
def get_region_name_by_code(code):
  region_name = {
        'CZ010': 'Hlavní město Praha',
        'CZ031': 'Jihočeský kraj',
        'CZ064': 'Jihomoravský kraj',
        'CZ041': 'Karlovarský kraj',
        'CZ052': 'Královéhradecký kraj',
        'CZ051': 'Liberecký kraj',
        'CZ080': 'Moravskoslezský kraj',
        'CZ071': 'Olomoucký kraj',
        'CZ053': 'Pardubický kraj',
        'CZ032': 'Plzeňský kraj',
        'CZ020': 'Středočeský kraj',
        'CZ042': 'Ústecký kraj',
        'CZ063': 'Vysočina',
        'CZ072': 'Zlínský kraj'
  }
  return region_name.get(code, 'Invalid regionCode')

# vývoj počtu nakažených v období 30.9.2020 - 30.10.2020 ve všech krajích ČR
for code in region_codes:
  number_of_cases_in_region = number_of_cases_by_regions_and_date.loc[number_of_cases_by_regions_and_date['kraj_nuts_kod'] == code]
  plt.figure(figsize=(22, 6))
  plt.plot(number_of_cases_in_region['datum'],number_of_cases_in_region['kumulativni_pocet_nakazenych'], color='g')
  plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False) # labels along the bottom edge are off
  plt.title('Vývoj počtu nakažených v ' + get_region_name_by_code(code), weight='bold', fontsize=16, color='white')
  plt.xlabel('Datum', weight='bold', fontsize=14, color='white')
  plt.ylabel('Kumulativní počet nakažených', weight='bold', fontsize=14, color='white')
  #plt.xticks(weight='bold', fontsize=10, rotation=45)
  plt.yticks(weight='bold', fontsize=10, color='white')
  plt.grid(color = 'black', linewidth = 0.5)

Příprava dat pro trénování sítě (definice target a features jakožto input proměnných do NN). <br>
Na základě analýzy dat (velmi podobný trend ve všech krajích) je pro trénování a validaci NN vytvořen dataset, který představuje vývoj v celé ČR (součty skrze kraje). <br>
Vstupní proměnné představují 14 (na základě studií) předešlých dnů pro aktuální den (target) a označují se jako timesteps - předešlé časové kroky jsou použity k predikci aktuálního časového kroku.


In [ ]:
# jako skutečné hodnoty pro daný den je použit součet pozitivních ve všech krajích (celorepublikový součet)
number_of_cases_all_regions = number_of_cases_by_regions_and_date.groupby(['datum'], as_index=False)[['kumulativni_pocet_nakazenych']].sum()

# datum je vhodné mít jako datetime nikoliv object
df_close = pd.DataFrame(number_of_cases_all_regions['kumulativni_pocet_nakazenych'])
df_close.index = pd.to_datetime(number_of_cases_all_regions['datum'])
print(number_of_cases_all_regions.dtypes)

# funkce pro vytvoření 14ti předešlých hodnot pro aktuální skutečnou hodnotu v daný den
def create_dataset_for_training(data, attribute, list_of_prev_instants):
  list_of_prev_instants.sort()
  start = list_of_prev_instants[-1] 
  end = len(data)
  data['datum'] = data.index
  data.reset_index(drop=True)

  new_data = data[start:end]
  new_data.reset_index(inplace=True, drop=True)
  # pro případ více atributů 
  for attribute in attribute :
          foobar = pd.DataFrame()

          for prev_instant in list_of_prev_instants :
              new_col = pd.DataFrame(data[attribute].iloc[(start - prev_instant) : (end - prev_instant)])
              new_col.reset_index(drop=True, inplace=True)
              new_col.rename(columns={attribute : '{}_(t-{})'.format(attribute, prev_instant)}, inplace=True)
              foobar = pd.concat([foobar, new_col], sort=False, axis=1)

          new_data = pd.concat([new_data, foobar], sort=False, axis=1)
     
  new_data.set_index(['datum'], drop=True, inplace=True)
  return new_data

# vytvoření datasetu pro trénink NN
attribute_name_for_training = ['kumulativni_pocet_nakazenych']
list_of_prev_cases_instants = []
for i in range(1,15):
    list_of_prev_cases_instants.append(i)
dataset_for_training = create_dataset_for_training(df_close, attribute_name_for_training, list_of_prev_cases_instants)
#dataset_for_training.head().to_csv(r'dataset_for_training.csv', index = False)

Rozdělení dat na trénovací, testovací a validační množinu. <br>
Testovací množina je tvořena novějšími daty - důvodem je predikce ve více regionech, mohlo by dojít k vytvoření nepřesného modelu, jenž není schopný reagovat na variabilitu v jednotlivých regionech (viz řešerše). Množina je tvořena 20% z celkového datasetu. <br>
Ze zbytku dat tvoří 90% trénovací data a zbylých 10% validační data. <br>
Jsou definovány testovací množiny pro predikování v jednotlivých krajích.



In [29]:
# velikosti množin
test_set_size = 0.20
valid_set_size = 0.10

# testujeme na aktuálnějších datech
dataset_for_training_copy = dataset_for_training.reset_index(drop=True)
dataset_test = dataset_for_training_copy.iloc[ int(np.floor(len(dataset_for_training_copy)*(1-test_set_size))) : ]
print(dataset_test.shape)

# starší zbylá data se rozdělí na trénovací a validační množinu
dataset_train_plus_valid = dataset_for_training_copy.iloc[ : int(np.floor(len(dataset_for_training_copy)*(1-test_set_size))) ]
dataset_train = dataset_train_plus_valid.iloc[ : int(np.floor(len(dataset_train_plus_valid)*(1-valid_set_size))) ]
dataset_valid = dataset_train_plus_valid.iloc[ int(np.floor(len(dataset_train_plus_valid)*(1-valid_set_size))) : ]

# rozdělení dle sloupců pro vstup do NN (target a timesteps jako vstupy do NN)
X_train, y_train = dataset_train.iloc[:, 1:], dataset_train.iloc[:, 0]
X_valid, y_valid = dataset_valid.iloc[:, 1:], dataset_valid.iloc[:, 0]
X_test, y_test = dataset_test.iloc[:, 1:], dataset_test.iloc[:, 0]

print('Shape of training inputs, training target:', X_train.shape, y_train.shape)
print('Shape of validation inputs, validation target:', X_valid.shape, y_valid.shape)
print('Shape of test inputs, test target:', X_test.shape, y_test.shape)

# funkce pro vytvoření testovací množiny pro predikci pro daný kraj
def create_test_set_for_prediction_by_region(code):
  # data pro daný kraj
  number_of_cases_in_region = number_of_cases_by_regions_and_date.loc[number_of_cases_by_regions_and_date['kraj_nuts_kod'] == code]

  # nechceme kopii
  new_df = number_of_cases_in_region.loc[:, number_of_cases_in_region.columns]

  # výpočet 14ti předešlých hodnot pro predikci
  dataset_region = create_dataset_for_training(new_df, attribute_name_for_training, list_of_prev_cases_instants)
  
  # definice testovacích dat - 10% dat
  test_set_region_size = 0.2
  dataset_region_test = dataset_region.iloc[ int(np.floor(len(dataset_region)*(1-test_set_region_size))) : ]
  return dataset_region_test

(20, 15)
Shape of training inputs, training target: (70, 14) (70,)
Shape of validation inputs, validation target: (8, 14) (8,)
Shape of test inputs, test target: (20, 14) (20,)


Škálování vstupních dat do NN v rozsahu 0.01 až 0.99 a úprava vstupních dat pro LSTM síť.

In [ ]:
# definice rozsahu škálování
Target_scaler = MinMaxScaler(feature_range=(0.01, 0.99))
Feature_scaler = MinMaxScaler(feature_range=(0.01, 0.99))

# aplikace škálování
X_train_scaled = Feature_scaler.fit_transform(np.array(X_train))
X_valid_scaled = Feature_scaler.fit_transform(np.array(X_valid))
X_test_scaled = Feature_scaler.fit_transform(np.array(X_test))
y_train_scaled = Target_scaler.fit_transform(np.array(y_train).reshape(-1,1))
y_valid_scaled = Target_scaler.fit_transform(np.array(y_valid).reshape(-1,1))
y_test_scaled = Target_scaler.fit_transform(np.array(y_test).reshape(-1,1))
print(X_test_scaled)
print(y_test_scaled)
# vstupní data pro LSTM síť (samples_number, timesteps_number, number_of_values_in_each_timestep)
# přidáme pouze jednu dimenzi (1 - máme 1 hodnotu v každé vstupní proměnné)
X_train_scaled_LSTM = np.expand_dims(X_train_scaled, 1)
X_valid_scaled_LSTM = np.expand_dims(X_valid_scaled, 1)
X_test_scaled_LSTM = np.expand_dims(X_test_scaled, 1)

Modely neuronových sítí:

1.   model je tradiční dopředná neuronová síť s 5 vrstvami:
    *   vstupní vrstva
    *   vrstva s 60 neurony a lineární aktivační funkcí
    *   vrstva s 60 neurony a lineární aktivační funkcí
    *   dropout vrstva s 25% ignorování neuronů - proti přeučení
    *   výstupní vrstva s lineární aktivační funkcí
  *   Optimalizační funkce: Adam a ztrátová funkce: Střední kvadratická chyba (MSE)
2.   model je LSTM síť s 5 vrstvami:
    *   vstupní vrstva s s délkou výstupu 14 (pamatuje si 14 hodnot)
    *   LSTM vrstva s délkou výstupu 14 (pamatuje si 14 hodnot) a lineární aktivační funkcí
    *   LSTM vrstva s 60 s délkou výstupu 14 (pamatuje si 14 hodnot) a lineární aktivační funkcí
    *   dropout vrstva s 25% ignorování neuronů - proti přeučení
    *   výstupní vrstva s lineární aktivační funkcí
  *   Optimalizační funkce: Adam a ztrátová funkce: Střední kvadratická chyba 



In [ ]:
# dopředná NN
#FNN_input_layer = Input(shape=(14), dtype='float32')
#FNN_dense1 = Dense(60, activation='linear')(FNN_input_layer)
#FNN_dense2 = Dense(60, activation='linear')(FNN_dense1)
#FNN_dropout_layer = Dropout(0.25)(FNN_dense2)
#FNN_output_layer = Dense(1, activation='linear')(FNN_dropout_layer)

#FNN_model = Model(inputs=FNN_input_layer, outputs=FNN_output_layer, name='dopredna_NN')
FNN_model = Sequential(name='dopredna_NN')

FNN_model.add(Input(shape=(14), dtype='float32', name='vstupni_vrstva'))

FNN_model.add(Dense(20, activation='linear', name='vrstva_1'))

FNN_model.add(Dropout(0.20, name='dropout_vrstva'))

FNN_model.add(Dense(1, activation='linear', name='vystupni_vrstva'))

FNN_model.compile(loss='mean_squared_error', optimizer='adam')

FNN_model.summary()

# LSTM síť
LSTM_model = Sequential(name='LSTM_NN')

LSTM_model.add(LSTM(units=20, return_sequences=True, input_shape=(X_train_scaled_LSTM.shape[1:]), name='LSTM_vrstva'))

LSTM_model.add(Dropout(0.20, name='dropout_vrstva'))

LSTM_model.add(Dense(units=1, activation='linear', name='vystupni_vrstva'))

LSTM_model.compile(optimizer='adam', loss='mean_squared_error')
LSTM_model.summary()
plot_model(LSTM_model)

Trénování sítí a jejich validace

In [ ]:
history_FNN_model = FNN_model.fit(x=X_train_scaled, y=y_train_scaled, batch_size=5, epochs=50, verbose=1, validation_data=(X_valid_scaled, y_valid_scaled), shuffle=True)
history_LSTM_model = LSTM_model.fit(X_train_scaled_LSTM, y_train_scaled, epochs=50, batch_size=5, verbose=1, validation_data=(X_valid_scaled_LSTM, y_valid_scaled), shuffle=True)

Průběh trénování obou modelů.

In [ ]:
# zobrazení průběhu FNN modelu
plt.figure(figsize=(20, 10))
plt.title('Průběh trénování FNN modelu', weight='bold', fontsize=16, color='white')
plt.xlabel('Počet epoch', weight='bold', fontsize=14, color='white')
plt.ylabel('Chyba', weight='bold', fontsize=14, color='white')
plt.xticks(weight='bold', fontsize=10, color='white')
plt.yticks(weight='bold', fontsize=10, color='white')
plt.plot(history_FNN_model.history['loss'], label = 'loss')
plt.plot(history_FNN_model.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show

# zobrazení průběhu LSTM modelu
plt.figure(figsize=(20, 10))
plt.title('Průběh trénování LSTM modelu', weight='bold', fontsize=16, color='white')
plt.xlabel('Počet epoch', weight='bold', fontsize=14, color='white')
plt.ylabel('Chyba', weight='bold', fontsize=14, color='white')
plt.xticks(weight='bold', fontsize=10, color='white')
plt.yticks(weight='bold', fontsize=10, color='white')
plt.plot(history_LSTM_model.history['loss'], label = 'loss')
plt.plot(history_LSTM_model.history['val_loss'], label = 'val_loss')
plt.legend()
plt.show

Uložení modelu pro použití ve springu na BE

In [53]:
FNN_model.save('FNN_model.h5')
LSTM_model.save('LSTM_model.h5')

Predikování na testovací množině pro celou ČR a jednotlivé kraje.

In [ ]:
# výpočet predikce na testovací množině pro celou ČR
!pwd 
y_pred_FNN = load_model('FNN_model.h5').predict(X_test_scaled)
y_pred_LSTM = load_model('LSTM_model.h5').predict(X_test_scaled_LSTM)
# úprava LSTM outputu (přebytečná dimenze pryč)
y_pred_LSTM_reshaped = np.squeeze(y_pred_LSTM, axis=1)
print(y_pred_LSTM_reshaped.shape)
# zpětné škálování z rozsahu 0.01-0.99 na původní hodnoty
y_pred_rescaled_FNN = Target_scaler.inverse_transform(y_pred_FNN)
y_pred_rescaled_LSTM = Target_scaler.inverse_transform(y_pred_LSTM_reshaped)
print(y_pred_rescaled_LSTM)

# funkce pro predikci v daném kraji (vracíme hodnotu v původním rozsahu)
def predict_in_region(code):
  dataset_region_test = create_test_set_for_prediction_by_region(code)

  # rozdělení na X a y
  X_test_region = dataset_region_test.iloc[:, 2:]
  y_test_region = dataset_region_test.iloc[:, 1]
  y_test_region_scaled = Target_scaler.fit_transform(np.array(y_test_region).reshape(-1,1))
  y_test_region_rescaled =  Target_scaler.inverse_transform(y_test_region_scaled)

  # škálování na rozsah 0.01 - 0.99
  X_test_region_scaled = Feature_scaler.fit_transform(np.array(X_test_region))
  X_test_region_scaled_LSTM = np.expand_dims( X_test_region_scaled, 1)

  # predikce FNN síťí
  y_pred_region_FNN = FNN_model.predict(X_test_region_scaled)
  y_pred_region_rescaled_FNN = Target_scaler.inverse_transform(y_pred_region_FNN)

  # predikce LSTM síťí
  y_pred_region_LSTM = LSTM_model.predict(X_test_region_scaled_LSTM)
  y_pred_region_LSTM_reshaped = np.squeeze(y_pred_region_LSTM, axis=1)
  y_pred_region_rescaled_LSTM = Target_scaler.inverse_transform(y_pred_region_LSTM_reshaped)

  return [y_pred_region_rescaled_FNN, y_pred_region_rescaled_LSTM, y_test_region_rescaled]

Posouzení kvality **FNN modelu** porovnáním predikovaných a aktuálních hodnot. <br>
Pro posouzení vybrán koeficient determinace R^2 - rozsah 0 (neužitečný model) - 1 (dokonalá predikce). <br>
Testováno nejprve pro celorepubliková data a následně pro všechny kraje.


In [ ]:
# zpětné škálování z rozsahu 0.01-0.99 na původní hodnoty
y_test_rescaled =  Target_scaler.inverse_transform(y_test_scaled)

# výpočet a vypsání R^2 pro celou ČR
score_CR = r2_score(y_test_rescaled, y_pred_rescaled_FNN)
print('Koeficient determinance pro testovací množinu - celá ČR:', round(score_CR,4))

# výpočet a vypsání R^2 pro všechny kraje
for code in region_codes:
  pred_region = predict_in_region(code)
  score_region = r2_score(pred_region[2], pred_region[0])
  print('Koeficient determinance pro testovací množinu - ' + get_region_name_by_code(code) + ':', round(score_region,4))

Grafické znázornění porovnání skutečných a predikovaných hodnot testovací množiny celorepublikově a v rámci všech krajů pro **FNN model**.

In [ ]:
# grafické porovnání v rámci ČR
plt.figure(figsize=(11, 6))
plt.plot(pd.DataFrame(y_test_rescaled), linestyle='solid', color='r')
plt.plot(pd.DataFrame(y_pred_rescaled_FNN), linestyle='dashed', color='b')

plt.legend(['Actual','Predicted'], loc='best', prop={'size': 14})
plt.title('Porovnání predikce a skutečných hodnot v celé ČR', weight='bold', fontsize=16, color='white')
plt.ylabel('Kumultaivní počet nakažených', weight='bold', fontsize=14, color='white')
plt.xlabel('Pořadí dnů testovací množiny', weight='bold', fontsize=14, color='white')
plt.xticks(weight='bold', fontsize=12, rotation=45, color='white')
plt.yticks(weight='bold', fontsize=12, color='white')
plt.grid(color = 'y', linewidth='0.5')
plt.show()

# grafické porovnání v rámci všech krajů
for code in region_codes:
  pred_region = predict_in_region(code)
  plt.figure(figsize=(11, 6))
  plt.plot(pd.DataFrame(pred_region[0]), linestyle='solid', color='r')
  plt.plot(pd.DataFrame(pred_region[2]), linestyle='dashed', color='b')

  plt.legend(['Actual','Predicted'], loc='best', prop={'size': 14})
  plt.title('Porovnání predikce a skutečných hodnot v ' + get_region_name_by_code(code), weight='bold', fontsize=16, color='white')
  plt.ylabel('Kumultaivní počet nakažených', weight='bold', fontsize=14, color='white')
  plt.xlabel('Pořadí dnů testovací množiny', weight='bold', fontsize=14, color='white')
  plt.xticks(weight='bold', fontsize=12, rotation=45, color='white')
  plt.yticks(weight='bold', fontsize=12, color='white')
  plt.grid(color = 'y', linewidth='0.5')
  plt.show()


Posouzení kvality **LSTM modelu** porovnáním predikovaných a aktuálních hodnot. Stejná metodika jako u FNN modelu.

In [ ]:
# výpočet a vypsání R^2 pro celou ČR
score_CR = r2_score(y_test_rescaled, y_pred_rescaled_LSTM)
print('Koeficient determinance pro testovací množinu - celá ČR:', round(score_CR,4))

# výpočet a vypsání R^2 pro všechny kraje
for code in region_codes:
  pred_region = predict_in_region(code)
  score_region = r2_score(pred_region[2], pred_region[1])
  print('Koeficient determinance pro testovací množinu - ' + get_region_name_by_code(code) + ':', round(score_region,4))

Grafické znázornění porovnání skutečných a predikovaných hodnot testovací množiny celorepublikově a v rámci všech krajů pro **LSTM model**.

In [ ]:
# grafické porovnání v rámci ČR
plt.figure(figsize=(11, 6))
plt.plot(pd.DataFrame(y_test_rescaled), linestyle='solid', color='r')
plt.plot(pd.DataFrame(y_pred_rescaled_LSTM), linestyle='dashed', color='b')

plt.legend(['Actual','Predicted'], loc='best', prop={'size': 14})
plt.title('Porovnání predikce a skutečných hodnot v celé ČR', weight='bold', fontsize=16, color='white')
plt.ylabel('Kumultaivní počet nakažených', weight='bold', fontsize=14, color='white')
plt.xlabel('Pořadí dnů testovací množiny', weight='bold', fontsize=14, color='white')
plt.xticks(weight='bold', fontsize=12, rotation=45, color='white')
plt.yticks(weight='bold', fontsize=12, color='white')
plt.grid(color = 'y', linewidth='0.5')
plt.show()

# grafické porovnání v rámci všech krajů
for code in region_codes:
  pred_region = predict_in_region(code)
  plt.figure(figsize=(11, 6))
  plt.plot(pd.DataFrame(pred_region[1]), linestyle='solid', color='r')
  plt.plot(pd.DataFrame(pred_region[2]), linestyle='dashed', color='b')

  plt.legend(['Actual','Predicted'], loc='best', prop={'size': 14})
  plt.title('Porovnání predikce a skutečných hodnot v ' + get_region_name_by_code(code), weight='bold', fontsize=16, color='white')
  plt.ylabel('Kumultaivní počet nakažených', weight='bold', fontsize=14, color='white')
  plt.xlabel('Pořadí dnů testovací množiny', weight='bold', fontsize=14, color='white')
  plt.xticks(weight='bold', fontsize=12, rotation=45, color='white')
  plt.yticks(weight='bold', fontsize=12, color='white')
  plt.grid(color = 'y', linewidth='0.5')
  plt.show()